In [1]:
# Aim: Scrape data from Wikipedia to create a text corpus
# Process:
# Result:

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
class WikiCategoryDataScrapper:
    def __init__(self, url):
        self.categoryURL = url
        self.categoryPageSoup = self.soup(self.request(self.categoryURL))
        self.index = self.letterIndices()
    
    def request(self,url):
        response = requests.get(url)
        return response

    def soup(self, response):
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    
    def letterIndices(self):
        index = {}
        soups = self.categoryPageSoup.body.find(id='mw-pages').find(class_='CategoryIndex').find_all('li')[2:]
        for soup in soups:
            index[soup.text] = soup.a['href']
        return index

In [4]:
url = 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&pageuntil=Essaim+de+drones#mw-pages'
scrapper = WikiCategoryDataScrapper(url)

In [5]:
scrapper.index

{'A': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=A',
 'B': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=B',
 'C': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=C',
 'D': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=D',
 'E': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=E',
 'F': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=F',
 'G': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=G',
 'H': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=H',
 'I': 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&from=I',
 